'Automatically' retires brownfield land endpoints. 

Needs a file `duplicates_list_file` per the code below, containing these fields: "resource", "endpoint", "ep_start_date", "ACTION", where the ACTION colun contains the letter R meaning Retire this one.

it reads the duplicates file and creates two files endpoint_new.csv and old-resource_new.csv

1. Runs through the above file, locating endpoints and setting their end-date to the dead-date (below).
1. Adds the resource to the old-resource file, setting their status to 410, meaning 'gone away'.


In [37]:
duplicates_list_file = r'/mnt/c/Users/MarkSmith/Downloads/Brownfield_Duplicates.csv'

source_folder= "/home/mark/DLUHC/brownfield-land-collection/collection/"

dead_date = "2023-07-24"


In [38]:
import sqlite3
import pandas as pd
import matplotlib 


In [41]:

all_duplicates = pd.read_csv(duplicates_list_file)

all_duplicates.head()

,Unnamed: 0,thiskey,resource,endpoint,dataset,res_start_date,organization,ep_start_date,end_date,ACTION,endpoint_url
0,0.0,Allerdale Borough Council_Brownfield land,3b694528538878d378fd6892a649fa634f0013ef299331...,5a0fcb2fdbe9d6f407b554642ab661a897a02c7a9e068a...,Brownfield land,2018-05-22,Allerdale Borough Council,2017-12-20,2019-11-25,NaN,https://www-cloudfront.allerdale.gov.uk/media/...
1,1.0,Allerdale Borough Council_Brownfield land,3b694528538878d378fd6892a649fa634f0013ef299331...,9c2e8adfd12b4f474e7d511580029d1e69d1e08a17e0cb...,Brownfield land,2018-05-22,Allerdale Borough Council,2017-12-20,NaN,R,https://df4iy9syor5px.cloudfront.net/media/fil...
2,2.0,Allerdale Borough Council_Brownfield land,d9d75707f0d600188137b494978bcd1b6fb6003f61495c...,734a30c885515b42eaa004258aede2f048e49ad136d28b...,Brownfield land,2023-07-08,Allerdale Borough Council,2023-07-06,NaN,NaN,https://www-cloudfront.allerdale.gov.uk/media/...
3,3.0,Amber Valley Borough Council_Brownfield land,93264a2648d3dbbf4a154681831e006fb7ee7bbec0a46a...,c0fcdc64525d3be69736427cd397bf5a3c2d68104e8a90...,Brownfield land,2018-05-22,Amber Valley Borough Council,2017-12-22,NaN,R,http://info.ambervalley.gov.uk/shareddatasets/...
4,4.0,Amber Valley Borough Council_Brownfield land,27710b83c46fb36c73ae7eb3d9011e7cd2488c535e04c7...,239f7a38efe51fb6271d40bd76dd7413fecb458a5c4d1e...,Brownfield land,2019-11-23,Amber Valley Borough Council,NaN,NaN,R,http://info.ambervalley.gov.uk/shareddatasets/...


In [42]:
# Load the files that make up the collection

old_resource_file = F"{source_folder}/old-resource.csv"
old_resource = pd.read_csv(old_resource_file)

endpoint_file =  F"{source_folder}/endpoint.csv"
endpoints = pd.read_csv(endpoint_file)

source_file = F"{source_folder}/source.csv"
source =  pd.read_csv(source_file)

In [43]:

duplicatea = all_duplicates [all_duplicates["ACTION"] == "R"]

duplicatea = duplicatea[["resource", "endpoint", "ep_start_date", "ACTION"]]

duplicatea.head()

,resource,endpoint,ep_start_date,ACTION
1,3b694528538878d378fd6892a649fa634f0013ef299331...,9c2e8adfd12b4f474e7d511580029d1e69d1e08a17e0cb...,2017-12-20,R
3,93264a2648d3dbbf4a154681831e006fb7ee7bbec0a46a...,c0fcdc64525d3be69736427cd397bf5a3c2d68104e8a90...,2017-12-22,R
4,27710b83c46fb36c73ae7eb3d9011e7cd2488c535e04c7...,239f7a38efe51fb6271d40bd76dd7413fecb458a5c4d1e...,NaN,R
5,ad2030e211cbb0f3b7b72d1739b25798620f8913d56158...,1945e57b32134a160bfb4972bcd6f86935b4c6fc0bd000...,NaN,R
6,317e0b502e4795ce0b03d1d0e787f8dbb7968d5755d884...,533d03f6fd2b5717597c4f3057106b907f4b23c8fb2e7b...,2020-05-21,R


In [47]:
for duplicate_row in duplicatea.iterrows():
    resource_value = duplicate_row[1].values[0]
    endpoint_value = duplicate_row[1].values[1]
    start_date_value = duplicate_row[1].values[2]
    
    print (F"{resource_value} // {endpoint_value} // {start_date_value}")

    # end it in endpint.csv
    endpoints.loc[endpoints.endpoint == endpoint_value, 'end-date'] = dead_date
    
    # write it to the old_resource file'
    old_resource.loc[len(old_resource)] = {'old-resource': resource_value, 'status': 410, 'resource': ''}


endpoint_file_out =  F"{source_folder}/endpoint_new.csv"
endpoints.to_csv (endpoint_file_out, index=False)

old_resource_file_out = F"{source_folder}/old-resource_new.csv"
old_resource.to_csv(old_resource_file_out, index=False)

3b694528538878d378fd6892a649fa634f0013ef299331e6c12b078cfe85d3b2 // 9c2e8adfd12b4f474e7d511580029d1e69d1e08a17e0cb1df754c2e77b6001ed // 2017-12-20
93264a2648d3dbbf4a154681831e006fb7ee7bbec0a46a4970285f4a847950e0 // c0fcdc64525d3be69736427cd397bf5a3c2d68104e8a9025a3f21662a804139c // 2017-12-22
27710b83c46fb36c73ae7eb3d9011e7cd2488c535e04c7bd0beb2587364d464b // 239f7a38efe51fb6271d40bd76dd7413fecb458a5c4d1e0febde7eb98550fb7b // nan
ad2030e211cbb0f3b7b72d1739b25798620f8913d5615859116de41f21a4ff8a // 1945e57b32134a160bfb4972bcd6f86935b4c6fc0bd0001dcf41353bb16d9041 // nan
317e0b502e4795ce0b03d1d0e787f8dbb7968d5755d88480437144d1243d4cc6 // 533d03f6fd2b5717597c4f3057106b907f4b23c8fb2e7b6d03fb900b20f97735 // 2020-05-21
2a88a5bb2cd54c35b964ca37ea4d21578d8ac8fa2c4b8cd08099b73eb413ed95 // a6b4129857b22b08d5e49133be22a4f07f34e89c1cb5d8d86a15e852f3435b5a // nan
3c12903110d1db128728a1dbd69069b6345b6db2a15afdb0ae6fc066eecfbcf5 // 683739b7f63a9e8f66e7868cbc02979cdcb3dbed3cf3fbd7ab8371590fd0c598 // nan